In [1]:
# Experiment 2 Script for COVID-19 Forecasting
# Change path to the root directory of the project
import os
os.chdir("../../")

# Description: This script contains the code for the second experiment in the project, 
# forecasting COVID-19 MVBeds using various RNN models and hyperparameter tuning with Simulated Annealing.

# Imports for handling data
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape

# Imports for visualization
import plotly.express as px
import plotly.graph_objects as go

# Progress bar
from tqdm.autonotebook import tqdm
tqdm.pandas()

# Local imports for data loaders and models
# from src.utils import plotting_utils
# from src.dl.dataloaders import TimeSeriesDataModule
# from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel, Seq2SeqConfig, Seq2SeqModel, RNNConfig
# from src.transforms.target_transformations import AutoStationaryTransformer

# Set seeds for reproducibility
pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

torch.set_float32_matmul_precision('high')

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import logging

# Set logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


Seed set to 42


In [2]:
# Utility Functions

def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig

def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample

def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)

# def mape(y_true, y_pred):
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def plot_forecast(pred_df, forecast_columns, selected_area, forecast_display_names=None, save_path=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.D3  # Use a colorblind-friendly palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines+markers",
            marker=dict(size=8, opacity=0.7, symbol='circle'),
            line=dict(color=act_color, width=3),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    line_styles = ["solid", "dash", "dot", "dashdot"]
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']
    for col, display_col, line_style, marker in zip(forecast_columns, forecast_display_names, cycle(line_styles), cycle(markers)):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=6, symbol=marker),
                line=dict(color=next(colors), width=2, dash=line_style),
                name=display_col,
            )
        )

    fig.update_layout(
        title=f"COVID-19 MVBeds Forecast Comparison for {selected_area}",
        title_font=dict(size=20),
        xaxis_title="Date",
        yaxis_title="COVID-19 MVBeds",
        xaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        yaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        legend=dict(
            font=dict(size=10),
            orientation="h",
            yanchor="top",
            y=1.02,  # Move the legend down to separate it from the title
            xanchor="center",
            x=0.5
        ),
        template="plotly_white",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=10, r=40, t=80, b=40),  # Adjust left margin for better alignment
        width=1200,
        height=600
    )

    if save_path:
        pio.write_image(fig, save_path)
    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

# Function to convert data to float32
def to_float32(df):
    for col in df.columns:
        df[col] = df[col].astype("float32")
    return df

In [3]:
# Load and Prepare Data
data_path = Path("data/processed/england_data.csv")
# data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data = pd.read_csv(data_path)
data["date"] = pd.to_datetime(data["date"])

# Select and Process Data
selected_area = "England"
# data_filtered = data[data["areaName"] == selected_area]
data.head()

,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,latitude,longitude,areaName
0,2020-04-01,0.0,23332,12059.0,3099,3989.0,694.0,35571.0,4730.0,56171302,1275584,474.050001,-12.133333,England
1,2020-04-02,1494.0,26264,12135.0,2932,3895.0,725.0,39466.0,5455.0,56171302,1275584,474.050001,-12.133333,England
2,2020-04-03,1788.0,28828,13635.0,2564,3878.0,737.0,43344.0,6192.0,56171302,1275584,474.050001,-12.133333,England
3,2020-04-04,1950.0,31421,15469.0,2593,3260.0,828.0,46604.0,7020.0,56171302,1275584,474.050001,-12.133333,England
4,2020-04-05,2097.0,34013,16657.0,2592,2994.0,823.0,49598.0,7843.0,56171302,1275584,474.050001,-12.133333,England


In [4]:
# Add time-lagged features
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f"_lag_{lag}"
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]
data, added_features = add_lags(data, lags, ["covidOccupiedMVBeds"])
data.dropna(inplace=True)


# Create temporal features
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data = create_temporal_features(data, "date")
data = data.set_index("date")
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 874 entries, 2020-04-22 to 2022-09-12
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   covidOccupiedMVBeds         874 non-null    float64
 1   cumAdmissions               874 non-null    int64  
 2   hospitalCases               874 non-null    float64
 3   newAdmissions               874 non-null    int64  
 4   new_confirmed               874 non-null    float64
 5   new_deceased                874 non-null    float64
 6   cumulative_confirmed        874 non-null    float64
 7   cumulative_deceased         874 non-null    float64
 8   population                  874 non-null    int64  
 9   openstreetmap_id            874 non-null    int64  
 10  latitude                    874 non-null    float64
 11  longitude                   874 non-null    float64
 12  areaName                    874 non-null    object 
 13  covidOccupiedMVB